In [48]:
import pandas as pd
pd.set_option('display.max_rows',500)
import numpy as np
from pymongo import MongoClient, UpdateOne
from datetime import datetime, timedelta
import glob, os, shutil
import time

In [2]:
import matplotlib.pyplot as plt

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

### MongoDB Conexion

In [31]:
conn = MongoClient("localhost:27017")
db = conn.get_database("expenses_control")

In [32]:
move = db.get_collection('movements')

In [33]:
def remove_docs():
    return move.remove({})

In [34]:
def collection_to_pd(collection):
    return pd.DataFrame(list(collection.find({},{'_id':0})))

In [187]:
def insert_single_object(collection):
    date = datetime.now().strftime("%d/%m/%Y")
    description = str(input('Describe the money flow briefly:'))
    quantity = float(input('Insert the correcto amount rounded to two decimal numbers:'))
    category = input('To what category does the movement belong?')
    return collection.insert_one({'date':date,
                            'description':description,
                            'quantity':quantity,
                            'category':category})

### Downloading and preparing bank data

In [35]:
USERNAME = "jpello6"
PASSWORD = "fapell6"

In [36]:
def data_extraction():
    
    os.environ['WDM_LOG_LEVEL'] = '0'
    browser = webdriver.Chrome(ChromeDriverManager().install())
    wait = WebDriverWait(browser, 10)
    browser.maximize_window()
    browser.get("https://bancaonline.bankinter.com/gestion/login.xhtml")
    browser.implicitly_wait(7)
    browser.find_element_by_xpath("//a[contains(text(),'aceptar')]").click()
 
    username = browser.find_element_by_name("uid")
    password = browser.find_element_by_name("password")
    
    username.send_keys(USERNAME)
    password.send_keys(PASSWORD)
    browser.find_element_by_id("btEntrar").click()
    browser.implicitly_wait(5)
    
    charges = wait.until(ec.element_to_be_clickable((By.XPATH, '/html/body/main/div/div/section/div[1]/form[1]/section/div/div[7]/div[1]/div[2]/div[3]/ul/li[1]')))
    charges.click()
    
    download_charges_excel = wait.until(ec.element_to_be_clickable((By.XPATH, '//html/body/main/section/form/div[1]/span[4]/div/ul/li[1]/a')))
    download_charges_excel.click()                                                                           
    
    return_home = wait.until(ec.element_to_be_clickable((By.XPATH, '//*[@id="menuCabecera"]/li[1]/a')))
    return_home.click()
    
    incomes = wait.until(ec.element_to_be_clickable((By.XPATH, '/html/body/main/div/div/section/div[1]/form[1]/section/div/div[1]/div[1]/div[2]/div[3]/ul/li[1]')))
    incomes.click()
    
    download_incomes_excel = wait.until(ec.element_to_be_clickable((By.XPATH, '/html/body/main/section/div[1]/span[1]/div[1]/form/ul/li[1]/a')))
    download_incomes_excel.click()

    
    time.sleep(5)

    browser.close()

In [37]:
def move_file():
    incomes_file = 'C:\\Users\\juanp\\Downloads\\movimientos.xls'
    charges_file = 'C:\\Users\\juanp\\Downloads\\Movimientos (1).xls'
    destination = 'C:\\Users\\juanp\\proyectos_varios\\expenses_control\\data'
    
    try:
        shutil.move(charges_file,destination)
        shutil.move(incomes_file, destination)
    except: 
        pass
    try:
        os.replace(charges_file, destination)
        os.replace(incomes_file, destination)
    except:
        pass

In [71]:
cash_out = ['PAGO BIZUM A','TRANS /Juan Maria Perez Lloren','CAJERO BANKINTER', 'COMISION EXCESO', 'CAJERO',
            'DZACARD','rmaffn','REDES INTERNACION','HUB']
cash_in = ['PAGO BIZUM DE', 'TRANSF NOMI','TRANS /PEREZ LLORENTE JUAN MAR']
transport = ['Wib Advance','ALSA','METRO DE','EasyPark','Cabify','Uber','RENFE','BLABLACAR','Busbud','SANTA ANA,S.L',
             'CARMEN PARDO VALCARC','ORIENTE','FLOWBIKES','AUTOPISTA','GPV', 'ARAFRE','EDREAMS','AirEuropa','TAXI','E.S.',
            'CEDIPSA']
shopping = ['MOONDAY','THEBRUBAKER','SUPERMERCADO','UNIDE','MARKET','SUPERCOR','DECATHLON','EXPRESS','CORTEFIEL','POLLOFRE',
           'ALBIE','FERRETERIA','PELUQUEROS','TATA GATEAU','MERCADONA','FERRETARIA','FARMACIA','SARRYMECA','ORIHUELA CAMPOAMOR']
restaurants = ['PEREGRINO','KUNFU CASTILLA','LA LLAMA','DIVINA DOLORES','TONELADS','RESTAURACION MARTI','CAFE'
               ,'NEW YORK BURGER','MARTINGALA','RESTAURANTE','HELADERIA','EL CHILITO',
              'TOPPINGS AND SALADS','BEFFCIOUS','TASTY POKE','SOTTGODIS','CORREQUESEQUEMA','BISTROTECA','ESPUMOSA'
               ,'SUPER ABUELA','SHUMARA','BAR DA PRAIA','HIPODROMO','EL PATIO MAHOU','TAIGA','TONELADAS','MOE', 'BAMBU CLUB',
              'MIMAYBE Y PONY', 'LA PEãUELA POZO', 'ADELA']
other_activities = ['CINESA','SINTRA', 'PLAYTOMIC','RECIB /CLUB DE PADEL Y TENIS F']
subscriptions = ['HBO','MEDIUM', 'CLUB DEPORTIVO','MICAPPITAL']
accomodation = ['LATAS SURF','HOSTELWORLD','SELINA']


def categorization(value):
    for cash in cash_out:
        if cash.lower() in value.lower():
            return 'cash-out'
        else:
            pass
    for vehicle in transport: 
        if vehicle.lower() in value.lower():
            return 'transport'
        else:
            pass
    for place in restaurants:
        if place.lower() in value.lower():
            return 'restaurant'
        else:
            pass
    for activity in other_activities:
        if activity.lower() in value.lower():
            return 'other activity'
        else:
            pass
    for subscription in subscriptions:
        if subscription.lower() in value.lower():
            return 'subscription'
        else:
            pass
    for stay in accomodation:
        if stay.lower() in value.lower():
            return 'accomodation'
        else:
            pass
    for shop in shopping:
        if shop.lower() in value.lower():
            return 'shopping'
        else:
            pass
    for cash in cash_in: 
        if cash.lower() in value.lower():
            return 'cash-in'

In [72]:
def insert_object(df, collection):
    '''
    This function inserts all the information from the dataframe we created with the previous function as a Mongodb object.
    This object will have two field: title (name of the film or show) and reviews (array with all the reviews users have made)
    '''

    collection.insert_many(df.to_dict('records'))

In [73]:
def bizum_categorization():
    date = str(input())
    name = str(input())
    qty = float(input())
    category = str(input())
    return move.update_one({'description':{'$regex':name},'date':date,
                                  'quantity':qty},{'$set':{'category':category}})
    

In [154]:
def data_insert(path_charges, path_incomes, collection):
    
    incomes = pd.read_excel(path_incomes)[3:]
    incomes.drop(['Unnamed: 1', 'Unnamed: 4'], axis = 1, inplace=True)
    incomes.rename(columns={'IBAN: ES6701280067740100026987':'date', 'Unnamed: 2':'description', 
                            'Unnamed: 3':'quantity'}, inplace=True)
    
    charges = pd.read_excel(path_charges)
    list_index = charges.index[charges["Número de tarjeta: Visa Clasi (....2133)"]=='Total Crédito'].tolist()
    for number in list_index:
        limit_index = number
    charges = charges[5:limit_index]
    charges.drop(['Unnamed: 2'], axis = 1, inplace=True)
    charges.rename(columns={'Número de tarjeta: Visa Clasi (....2133)':'date', 'Unnamed: 1':'description', 
                            'Unnamed: 3':'quantity'}, inplace=True)
    charges['date'] = charges['date'].apply(lambda x: x.strftime('%d/%m/%Y'))
    
    complete = pd.concat([incomes, charges])
    
    current_month = datetime.today().date().replace(day=1).strftime('%m/%Y')
    first_day_prev_month = datetime.today().replace(day=1) - timedelta(days=1)
    prev_month = first_day_prev_month.strftime('%m/%Y')
    complete.drop(complete[complete.description == 'RECIBO VISA CLASICA'].index, inplace = True)
    
    old = pd.DataFrame(list(move.find({'date':{'$regex':current_month}},{'_id':0,'category':0})))
    
    complete_up = complete.loc[complete['date'].str.contains(current_month)]
    final_df = pd.concat([old, complete_up])
    final_df.drop_duplicates(keep = False, inplace = True)
    final_df['category'] = list(map(categorization, final_df['description']))
    
    option = input('What action do you want to take? - update or insert').lower()
        
    if option == 'insert':
        insert_object(complete, move)
        return 'Data succesfully inserted'

    elif option == 'update':
        if len(final_df) == 0:
            return 'Nothing to update'
        else:
            return final_df
            ##insert_object(final_df, move)
            return 'Data succesfully updated'

In [155]:
def my_bank_movements():
    
    try:
    
        data_extraction()
        
    except:
        
        return 'There has been a problem with the conexion. Please, try again later'
    
    move_file()

    return data_insert('data\\movimientos.xls', 'data\\Movimientos (1).xls', move)
    